In [ ]:
from google.colab import drive
import zipfile
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/HLCV_Project/archive.zip -d /data

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt


# Define the data directory paths
data_dir = '/data'
test_dir = os.path.join(data_dir, 'test')

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from PIL import Image

# Define the dataset class for labeled CIFAKE images
class LabeledCIFAKEDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        for label, folder in enumerate(['REAL', 'FAKE']):
            folder_path = os.path.join(root_dir, folder)
            for img_name in os.listdir(folder_path):
                self.images.append(os.path.join(folder_path, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define the binary classification model
class BinaryClassificationResNet(nn.Module):
    def __init__(self, pretrained_path):
        super(BinaryClassificationResNet, self).__init__()
        self.resnet = models.resnet50()

        # Modify the first convolutional layer to accept 3 channels and remove maxpooling layer
        self.resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet.maxpool = nn.Identity()

        # Load pretrained weights with strict=False to ignore mismatches
        pretrained_dict = torch.load(pretrained_path)
        model_dict = self.resnet.state_dict()

        # 1. Filter out unnecessary keys
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # 2. Overwrite entries in the existing model state dict
        model_dict.update(pretrained_dict)
        # 3. Load the new state dict
        self.resnet.load_state_dict(model_dict, strict=False)

        # Replace the final fully connected layer for binary classification
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 2)

    def forward(self, x):
        return self.resnet(x)

# Function to test the model on the test dataset
def test_binary_classifier(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    test_predictions = []
    test_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

            test_predictions.extend(predicted.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_accuracy = 100. * test_correct / test_total
    precision = precision_score(test_labels, test_predictions, average='binary')
    recall = recall_score(test_labels, test_predictions, average='binary')
    test_f1 = f1_score(test_labels, test_predictions, average='binary')

    print(f'Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {test_f1:.4f}\n')

# Main execution for testing
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create test dataset and dataloader
    test_dataset = LabeledCIFAKEDataset(root_dir=test_dir, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

    # Initialize the model
    pretrained_path = '/content/drive/MyDrive/HLCV_Project/rotation_resnet_cifake.pth'  # Path to the pretrained model from the pretext task
    model = BinaryClassificationResNet(pretrained_path).to(device)

    # Load the best model weights
    model.load_state_dict(torch.load('/content/drive/MyDrive/HLCV_Project/best_bin_class.pth'))

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    # Test the model
    test_binary_classifier(model, test_loader, criterion, device)

if __name__ == "__main__":
    main()
